### Example Usage
(of the `embedding_models.py` file.)
The file exposes a `ModelType` enum and an `embed_inputs` function which takes in a `pd.Series`, a model type and returns a new `pd.Series` representing the embedding for an input text using the method specified.

The function is expecting the `pd.Series` to be a list of strings, rather than a list of list of strings. 

In [5]:
import importlib
from embedding_models import embed_inputs, ModelType
import pandas as pd

In [11]:
df = pd.DataFrame(
    {
        "text": [
            "Sweet nutty flavours",
            "This is a different sentence",
            "Piss yellow beer",
            "Not sweet enough",
        ]
    }
)
for model in ModelType:
    df[model.name] = embed_inputs(df["text"], model=model)
df.head()

,text,SentenceTransformer,BERT,Count,TFIDF
0,Hazy nutty flavours,"[-0.05412261560559273, -0.06606823951005936, 0...","[-0.24941563606262207, -0.44494691491127014, 0...","[0, 0, 1, 1, 0, 1, 0, 0, 0, 0]","[0.0, 0.0, 0.5773502691896257, 0.5773502691896..."
1,This is another test,"[0.02996388077735901, -0.024454835802316666, -...","[-0.12034142762422562, -0.6497827172279358, 0....","[1, 0, 0, 0, 1, 0, 0, 1, 1, 0]","[0.5, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.5, ..."
2,Piss yellow beer,"[-0.03343258053064346, 0.0571899488568306, -0....","[-0.07380929589271545, 0.6937302947044373, 0.0...","[0, 1, 0, 0, 0, 0, 1, 0, 0, 1]","[0.0, 0.5773502691896257, 0.0, 0.0, 0.0, 0.0, ..."


##### An example of similarity

We can now comput cosine similarity between pairs of sentences as below.

In [13]:
from sentence_transformers import util
import torch

methods = [modeltype.name for modeltype in ModelType]
df_methods = pd.DataFrame(index=methods)

for model in ModelType:
    for i in range(1, len(df)):
        df_methods.loc[model.name, i + 1] = util.cos_sim(
            torch.Tensor(df[model.name][0]), torch.Tensor(df[model.name][i])
        ).item()

print("Texts: " + str(df["text"].values))
print("Similarity between first and nth sentence:")
df_methods.head()

Texts: ['Hazy nutty flavours' 'This is another test' 'Piss yellow beer']
Similarity between first and nth sentence:


,2,3
SentenceTransformer,0.153935,0.314107
BERT,0.562694,0.763873
Count,0.000000,0.000000
TFIDF,0.000000,0.000000
